In [1]:
import os
import sys
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

from itertools import cycle
from matplotlib import pyplot as plt

from simulation import SimulationMixin


In [2]:
fontsize = 20

plots_dir = "plots"

In [3]:
colors = [("red", "red"), ("green", "green"), ("blue", "blue"), ("orange", "orange")]
color_cycle = cycle(colors)

In [ ]:
program = "coloring"
# program = "maximal_matching"
# program = "dijkstra"
# graph_name = "graph_powerlaw_cluster_graph_n7"
# graph_name = "implicit_graph_n8"
graph_name = "graph_10"
sched = 0
no_simulations = 500000
me = False
fault_interval = 1

duong_mode = True

selected_nodes = [0, 4, 7, 9]

In [5]:
def get_filename(graph_name, sched, simulation_type, args, no_simulations, me, fault_interval):
    return f"{graph_name}__{sched}__{simulation_type}_args_{args}__{no_simulations}__{me}__{fault_interval}"

In [6]:
filenames = [
    get_filename(
        graph_name,
        sched,
        SimulationMixin.RANDOM_FAULT_SIMULATION_TYPE,
        "",
        no_simulations,
        me,
        fault_interval,
    )
]
if duong_mode:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE_DUONG,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )
else:
    filenames.extend(
        [
            get_filename(
                graph_name,
                sched,
                SimulationMixin.CONTROLLED_FAULT_AT_NODE_SIMULATION_TYPE,
                arg,
                no_simulations,
                me,
                fault_interval,
            )
            for arg in selected_nodes
        ]
    )


In [7]:
dfs = [pd.read_csv(os.path.join("results", program, f"{fn}.csv")) for fn in filenames]

In [8]:
max_steps = max(df['Steps'].max() for df in dfs)
max_steps

np.int64(13)

In [9]:
bins = np.linspace(0, max_steps, max_steps+1)
# bin_centers = (bins[:-1] + bins[1:]) / 2
bin_centers = bins

In [10]:
bins

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11., 12.,
       13.])

In [11]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_data[0][0]

array([    0, 35447, 72122, 73274, 74128, 73105, 66074, 50560, 31985,
       15823,  5761,  1480,   241])

In [12]:
hist_data = [np.histogram(df['Steps'], bins=bins) for df in dfs]
hist_df = []
for hd in hist_data:
    hist_df.append(
        pd.DataFrame({
            'Steps': bin_centers[:-1],
            'Count': hd[0]
        })
    )

In [13]:
hist_df[0]

,Steps,Count
0,0.0,0
1,1.0,35447
2,2.0,72122
3,3.0,73274
4,4.0,74128
5,5.0,73105
6,6.0,66074
7,7.0,50560
8,8.0,31985
9,9.0,15823


In [14]:
df_merged = hist_df[0]
for i in range(1, len(hist_df)):
    df_merged = pd.merge(df_merged, hist_df[i], on=["Steps"], suffixes=(i-1, i))
df_merged

,Steps,Count0,Count1,Count2,Count3
0,0.0,0,0,0,0
1,1.0,35447,4749,37920,37708
2,2.0,72122,76766,71616,71751
3,3.0,73274,78035,72933,72938
4,4.0,74128,79340,74136,74422
5,5.0,73105,77655,72665,72828
6,6.0,66074,69862,65272,65125
7,7.0,50560,54244,50461,50278
8,8.0,31985,34256,31937,31906
9,9.0,15823,17056,15528,15497


In [15]:
df_merged.set_index('Steps', drop=True, inplace=True)
df_merged

,Count0,Count1,Count2,Count3
Steps,,,,
0.0,0,0,0,0
1.0,35447,4749,37920,37708
2.0,72122,76766,71616,71751
3.0,73274,78035,72933,72938
4.0,74128,79340,74136,74422
5.0,73105,77655,72665,72828
6.0,66074,69862,65272,65125
7.0,50560,54244,50461,50278
8.0,31985,34256,31937,31906


In [16]:
def get_title():
    return f"Simulation - {program} | {graph_name} | Sched: {sched} | N: {no_simulations:,} | FI: {fault_interval}"

In [17]:
def get_filename():
    return f"{graph_name}__{sched}__{no_simulations:,}__{fault_interval}__{''.join([str(i) for i in selected_nodes])}{'__duong' if duong_mode else ''}"

In [18]:
plt.figure(figsize=(16, 8))
ax = sns.lineplot(data=df_merged, marker="o", linewidth=3)
ax.set_title(get_title(), fontdict={"fontsize": fontsize})

# ax.set_yscale("log")
for i, line in enumerate(ax.lines):
    if i >= 1:
        line_style = "solid"
        line.set_color(next(color_cycle)[0])
    else:
        line_style = "dashed"
        line.set_color("goldenrod")
    line.set_linestyle(line_style)

ax.tick_params(axis="x", labelsize=fontsize)
ax.tick_params(axis="y", labelsize=fontsize)
ax.xaxis.label.set_size(fontsize)
ax.yaxis.label.set_size(fontsize)

ax.set_xlabel("Steps")
ax.set_ylabel("Count")

labels = ["Random Fault"]
labels.extend(
    [
        f'Controlled {"(duong)" if duong_mode else ""} at node {n}'
        for n in selected_nodes
    ]
)
custom_lines = [
    mlines.Line2D(
        [],
        [],
        color=line.get_color(),
        # marker=marker,
        label=cat,
        linewidth=1,
        linestyle=line.get_linestyle(),
    )
    for line, cat in zip(ax.lines, labels)
]
plt.rc("font", size=fontsize)
plt.legend(handles=custom_lines, fontsize=fontsize * 0.9)
file_path = os.path.join(
    plots_dir,
    program,
    f"{get_filename()}.png",
)
plt.savefig(
    file_path,
    bbox_inches="tight",
)
plt.close()

print(f"Saved plot(s) for {file_path}")

Saved plot(s) for plots/coloring/star_graph_n13__0__500,000__1__059__duong.png
